In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

driver = webdriver.Chrome()
driver.get("https://slmis.xu.edu.ph/")

# OPEN THE GRADEBOOK BEFORE RUNNING SUCCEEDING

In [7]:
driver.switch_to.frame(driver.find_element(By.NAME, "TargetContent"))
table = driver.find_element(By.XPATH, '//table[@role="presentation"]')
inputs = table.find_elements(By.TAG_NAME, "input")
element_names = [i.get_attribute("id") for i in inputs if "DERIVED_LAM_GRADE" in i.get_attribute("id")]
students = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "HCR_PERSON_NM_I_NAME" in i.get_attribute("id")]
desc = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "DERIVED_LAM_ASSIGNMENT_DESCR" in i.get_attribute("id")]

In [8]:
df = pd.DataFrame({'Names':students})
for i in desc:
    df[i] = ""

df.to_excel("format.xlsx", index=False)

In [9]:
saved_df = pd.read_excel("b3_1.xlsx")
for name in tqdm(element_names):
    try:
        col, row = [int(i) for i in name[18:].split('$')]
        grade = saved_df.iloc[row, col]
        ActionChains(driver) \
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .perform()
        
    except StaleElementReferenceException:
        print("StaleElementReferenceException")
        time.sleep(1)
        ActionChains(driver) \
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .perform()

print("Successfully imported all final grades.\n Please click on save")

    # action = ActionBuilder(driver)
    # action.pointer_action.move_to_location(8, 0)
    # action.perform()

  0%|          | 0/245 [00:00<?, ?it/s]

  3%|▎         | 7/245 [00:07<04:35,  1.16s/it]

StaleElementReferenceException


  7%|▋         | 16/245 [00:20<04:50,  1.27s/it]

StaleElementReferenceException


...

StaleElementReferenceException


 91%|█████████▏| 224/245 [04:53<00:25,  1.22s/it]

StaleElementReferenceException


100%|██████████| 245/245 [05:17<00:00,  1.29s/it]

Successfully imported all final grades.
 Please click on save


In [2]:
len("DERIVED_LAM_GRADE")

17